Import ```SimPy``` and ```Random``` libraries to perform the simulation and generate random numbers. Import ```matplotlib.pylot``` to visualize the outcome.

In [ ]:
import simpy
import random
import matplotlib.pyplot as plt

Initialize a set of globals that define the characteristics of the model instance to be simulated. This includes the lower and upper bound for interarrival times, shape $k$ and rate  $\lambda$ for [Erlang distribution](https://en.wikipedia.org/wiki/Erlang_distribution) (station-1 service times), rate $\lambda$ for [exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) (reneging times and station 2 service times). 

In [ ]:
RANDOM_SEED = 978
A = 4
B = 10
STATION1_SHAPE = 3
STATION1_RATE = 0.468
RENEGE_RATE = 0.04
STATION2_RATE = 0.03
STATION2_SERVER_COUNT = 4
random.seed(RANDOM_SEED)

print("Expected interarrival times: ", (A+B)/2)
print("Expected renege time: ", 1/RENEGE_RATE)
print("Expected station #1 service time: ", round(STATION1_SHAPE/STATION1_RATE, 2))
print("Expected station #2 service time: ", round(1/STATION2_RATE, 2))

Define the necessary set of arrays for recording different information about the nature of a simulation.

In [ ]:
station1_service_times = []            # service times for station 1
queue_w_times = []                     # Time spent by a job while it waits for station 1
inter_station_w_times = []             # Time spent by a job while it waits for station 2 (blocking station1)
station2_service_times = []            # service times for station 2
jobs_that_renege = []                  # number of jobs left the system without being served
sojourn_time = []                      # sojourn time for each job
renege_times = []                      # renege time
interarrival_times = []                # interrarival times
number_of_people = {}                  # number of people in the system at timestamp t
simulation_end = 0

def clear_all():
    station1_service_times.clear()
    queue_w_times.clear()
    inter_station_w_times.clear()
    station2_service_times.clear()
    jobs_that_renege.clear()
    sojourn_time.clear()
    renege_times.clear()
    interarrival_times.clear()
    number_of_people.clear()
    simulation_end = 0

At the end of the simulation display total service time of station #1, total waiting time in the queue, average sojourn time per job, proportion of time station #1 was blocked, average reneging rate and proportion of jobs that completed the service. ```Statistics``` lists are created to store the necessary information throughout the simulations.

In [ ]:
# Statistics lists

sojourn = []
renege = []  # average rate (average number of people who reneged)
block = []
mean_interarrival = []
mean_station1_service = []
mean_station2_service = []
mean_queue_wait = []
renege_time = []  # average renege time
jobs_completed = []
mean_number_of_jobs = []
people = 0

def clear_statistics():
    sojourn.clear()
    renege.clear()
    block.clear()
    mean_interarrival.clear()
    mean_station1_service.clear()
    mean_station2_service.clear()
    mean_queue_wait.clear()
    renege_time.clear()
    jobs_completed.clear()
    mean_number_of_jobs.clear()
    people = 0

In [ ]:
# Display the statistical outcome of a simulation and append the outcomes to the storage for statistical analysis

def display_stats(size):
    mean_interarrival.append( (round(sum(interarrival_times)/len(interarrival_times), 3)))
    renege_time.append(  round(sum(renege_times)/len(renege_times), 3))
    mean_station1_service.append(  round(sum(station1_service_times)/len(station1_service_times), 3)) 
    mean_station2_service.append(  round(sum(station2_service_times)/len(station2_service_times), 3))
    mean_queue_wait.append(  round(sum(queue_w_times)/len(queue_w_times), 3))
    sojourn.append(  round(sum(sojourn_time) / size, 3))
    renege.append(  round(len(jobs_that_renege)/ size, 3))
    block.append(  round( sum(inter_station_w_times) / simulation_end,3))
    jobs_completed.append(  round( ((size - len(jobs_that_renege))/size), 3))
    
    sum_ = 0
    keys = list(number_of_people.keys())
    for i in range(len(keys) - 1):
        sum_ = sum_ + (keys[i+1] - keys[i]) * number_of_people[keys[i]] 
    mean_number_of_jobs.append( round(sum_/simulation_end, 3))
    
    print()
    print("Statistics:")
    print("Average interarrival time:", mean_interarrival[-1])
    print("Average renege time:", renege_time[-1])
    print("Total station #1 service time: ", round(sum(station1_service_times), 3))
    print("Station #1 average service time: ", mean_station1_service[-1])
    print("Station #2 average service time: ", mean_station2_service[-1])
    print("Average waiting time in the queue of station #1: ",mean_queue_wait[-1])
    print("Average sojourn time: ", sojourn[-1])
    print("Proportion of time the server was blocked: ", block[-1])
    print("Average reneging rate: ", renege[-1])
    print("Proportion of jobs that completed the service: ", jobs_completed[-1])
    print("Average number of jobs in the system per unit time: ", mean_number_of_jobs[-1])
    print("Little's Law - Calculated value of average number of jobs: ", round(sojourn[-1] / mean_interarrival[-1], 3))
    print()

In [ ]:
# job arrive one by one at a queueing network.
class job(object):
    def __init__(self, name, env, station1, station2):
        self.env = env
        self.name = name                            # index of the job
        self.arrival_t = self.env.now               # arrival time
        self.station1 = station1
        self.station2 = station2
        action = self.env.process(self.station())      
        
    def station(self):
        global people
        env = self.env
        print('Timestamp: %.2f -- %s has entered the system.' % (self.env.now, self.name)) 
        people += 1
        number_of_people[self.env.now] = people
        reneged = False
        with self.station1.request() as req:
            # Check if queue is empty. Move forward to the server if it's empty, otherwise enter the queue.
            if len(self.station1.queue) > 0:
                patience = next_renege_time()
                renege_times.append(patience)
                
                # wait for the service or abort at the end of our tether
                results = yield req | self.env.timeout(patience)
                wait = self.env.now - self.arrival_t
            else: 
                wait = 0   # no wait
                yield req
            
            if wait == 0 or req in results: # we got to the server
                if(wait > 0): print('Timestamp: %.2f -- %s waited for %.2f seconds in the queue, now it enters station #1 server.' % (self.env.now, self.name, wait))
                else: print('Timestamp: %.2f -- %s enters station #1 server without waiting in the queue.' % (self.env.now, self.name))
                
                queue_w_times.append(wait)
                
                # now being served
                yield self.env.process(self.served1())
                        
            else:  # we renege
                print('Timestamp: %.2f -- %s reneged after waiting %.2f seconds in the queue.' % (self.env.now, self.name, wait))
                people -= 1
                number_of_people[self.env.now] = people
                jobs_that_renege.append(self.name)
                queue_w_times.append(wait)
                reneged = True
     
        
        if(reneged):  
            sojourn_time.append(self.env.now - self.arrival_t)
        else:
            # now we have to proceed to station2
            with self.station2.request() as req:
                # It's guarantee that there is one server idle in the station #2.        
                yield self.env.process(self.served2())
                
                print('Timestamp: %.2f -- %s leaves the system.' % (self.env.now, self.name))
                people -= 1
                number_of_people[self.env.now] = people
                sojourn_time.append(self.env.now - self.arrival_t)
                
                global simulation_end
                simulation_end = self.env.now
         
    def served1(self):
        duration = next_station1_service_time()
        yield self.env.timeout(duration)  # waiting for service to be completed
        print('Timestamp: %.2f -- %s completes its operation in station #1 server.' % (self.env.now, self.name))
        station1_service_times.append(duration)   # keep record of the service times
        
        station1_left = self.env.now
        with self.station2.request() as req:
            yield req
            station2_enter = self.env.now
            block = station2_enter - station1_left
            inter_station_w_times.append(block)
            if(block == 0) : print('Timestamp: %.2f -- %s enters station #2 server immediately after finishing its job in station #1.' % (self.env.now, self.name))
            else : print('Timestamp: %.2f -- %s blocked station #1 for %.2f seconds, now it enters station #2 server.' % (self.env.now, self.name, block))
            
    
    def served2(self):
        duration =  next_station2_service_time()
        yield self.env.timeout(duration) # waiting for service to be completed
        station2_service_times.append(duration)

Represent the jobs in the station #2 at the beginning of the simulation.

In [ ]:
# immediately starts from station 2
class job_station_2(object):
    def __init__(self, name, env, station1, station2):
        self.env = env
        self.name = name                            # index of the job
        self.arrival_t = self.env.now               # arrival time
        self.station1 = station1
        self.station2 = station2

        action = env.process(self.station())      
        
    def station(self): 
        global people
        print('Timestamp: %.2f -- %s has entered the system and directly goes to station #2.' % (self.env.now, self.name))
        people += 1
        number_of_people[self.env.now] = people
        # now we have to proceed to station2
        with self.station2.request() as req:
            # It's guarantee that there is one server idle in the station #2.        
            yield self.env.process(self.served2())
                
            print('Timestamp: %.2f -- %s leaves the system.' % (self.env.now, self.name))
            people -= 1
            number_of_people[self.env.now] = people
            sojourn_time.append(self.env.now - self.arrival_t)
                
            global simulation_end
            simulation_end = self.env.now
    
    def served2(self):
        duration =  next_station2_service_time()
        yield self.env.timeout(duration) # waiting for service to be completed
        station2_service_times.append(duration)  

In [ ]:
# generate the arrivals
def job_generator(env, size, idx, station1 , station2):
    """Generate new jobs that arrive at the system."""
    for _ in range(size):
        interarrival = next_interarrival_time()
        interarrival_times.append(interarrival)
        yield env.timeout(interarrival) # wait until arrival time passes
        # now at this point, job arrives at the system
        Job = job('Job %s' %(idx), env, station1, station2)
        idx += 1

#### 0) Test the hand simulation with pre-generated random numbers.

In [ ]:
arr = iter([6.34, 8.68, 4.36, 8.74, 9.76, 6.7, 4.54, 5.08, 6.64, 5.86, 7.66, 6.82, 7.9, 7.9, 5.32])
rng = iter([51.01, 10.77, 14.95, 55.18, 12.36, 33.68, 70.34, 18.35, 13.62, 5.27, 80.47, 6.21])   # some are missing since they don't enter queue
st1 = iter([6.42, 6.28, 9.72, 5.14, 9.75, 3.49, 5.48, 5.92, 5.10, 9.1, 2.31, 2.73, 8.61])           # 13 and 15 are missing since they renege 
st2 = iter([0.67, 2.78, 9.59, 18.16, 14.36, 84.19, 53.65, 39.04, 8.28, 17.03, 63.24, 11.89, 25.88]) # 13 and 15 are missing since they renege

def next_interarrival_time():
    return next(arr)

def next_station1_service_time():
    return next(st1)

def next_renege_time():
    return next(rng)

def next_station2_service_time():
    return next(st2)

In [ ]:
def test_hand_simulation(size):    
    clear_all()  # clear the lists
    env = simpy.Environment()
    station1 = simpy.Resource(env, capacity = 1)  # station1 with capacity = 1
    station2 = simpy.Resource(env, capacity = 4) # station2 with capacity = 4
    env.process(job_generator(env, size = size, idx = 1, station1= station1, station2 = station2))
    env.run() 

size = 15
test_hand_simulation(size)
display_stats(size)



#### Random Number Generation
Necessary functions to provide random numbers. Refer to [docs](https://docs.python.org/3/library/random.html) for details.

In [ ]:
def next_interarrival_time():
    return random.uniform(A, B)

def next_station1_service_time():
    return random.gammavariate(alpha = STATION1_SHAPE, beta = 1 /STATION1_RATE)

def next_renege_time():
    return random.expovariate(RENEGE_RATE)

def next_station2_service_time():
    return random.expovariate(STATION2_RATE)

**To Do**: Run all your simulations with the following three starting conditions:
1) with an empty queueing network.<br>
2) with 5 jobs in the first system and half of the c servers full (if not divisible by 2 round down) in the second system.<br>
3) with 10 jobs in the first system and all of the c servers full in the second system.<br><br>
For each condition run each simulation for 20, 200 and 1000 exiting jobs using the same random number seed.<br>
Change the random number seed and repeat the previous runs.

#### 1) Run the simulation with an empty queueing network. ####

In [ ]:
def simulation_1(jobs, random_seed = RANDOM_SEED):        
    clear_statistics()
    for idx, j in enumerate(jobs):
        if(type(random_seed) == list): random.seed(random_seed[idx])
        else : random.seed(random_seed)
            
        print("Simulation 1 - exiting jobs:", j)
        clear_all()  # clear the lists to store statistics
        env = simpy.Environment()
        station1 = simpy.Resource(env, capacity = 1)  # station1 with capacity = 1
        station2 = simpy.Resource(env, capacity = 4)  # station2 with capacity = 4
        env.process(job_generator(env, size = j, idx = 1, station1 = station1, station2 = station2))
        env.run() 
        print()
        display_stats(j)
        print("------------" * 10)

simulation_1(jobs = [20, 200, 1000])

#### 2) Run the simulation with 5 jobs in the first system and half of the c servers full (if not divisible by 2 round down) in the second system. ####

In [ ]:
def simulation_2(jobs, random_seed = RANDOM_SEED):
    print(RANDOM_SEED)
    clear_statistics()
    for j in jobs:
        random.seed(random_seed)
        clear_all()
        env = simpy.Environment()                     # set up simulation environment (includes time clock, event list etc.)
        station1 = simpy.Resource(env, capacity = 1)  # station1 with capacity = 1
        station2 = simpy.Resource(env, capacity = 4 ) # station2 with capacity = 4
        # generate five jobs in the first system
        idx = 1
        for _ in range(5):
            Job = job('Job %s' %(idx), env, station1, station2)
            idx += 1

        # generate two jobs in the second system
        for _ in range(STATION2_SERVER_COUNT // 2):
            Job = job_station_2('Job %s' %(idx), env, station1, station2)
            idx += 1

        env.process(job_generator(env, size = j, idx = idx, station1 = station1, station2 = station2)) # generate j many jobs with interarrival rates
        env.run() 

        display_stats(j + 7) # 5 jobs in the first station, 2 jobs in the second station
        
simulation_2(jobs = [13, 193, 993]) # run the simulation with RANDOM_SEED

#### 3) Run the simulation with 10 jobs in the first system and all of the c servers full in the second system. ####

In [ ]:
def simulation_3(jobs, random_seed = RANDOM_SEED):
    print(RANDOM_SEED)
    clear_statistics()
    for j in jobs:
        random.seed(random_seed)
        clear_all()
        env = simpy.Environment()                     # set up simulation environment (includes time clock, event list etc.)
        station1 = simpy.Resource(env, capacity = 1)  # station1 with capacity = 1
        station2 = simpy.Resource(env, capacity = 4 ) # station2 with capacity = 4
        # generate five jobs in the first system
        idx = 1
        for _ in range(10):
            Job = job('Job %s' %(idx), env, station1, station2)
            idx += 1

        # generate two jobs in the second system
        for _ in range(STATION2_SERVER_COUNT):
            Job = job_station_2('Job %s' %(idx), env, station1, station2)
            idx += 1

        env.process(job_generator(env, size = j, idx = idx, station1 = station1, station2 = station2)) # generate j many jobs with interarrival rates
        env.run() 

        display_stats(j + 15) # 10 jobs in the first station, 5 jobs in the second station
        
simulation_3(jobs = [6, 186, 986]) # run the simulation with RANDOM_SEED

#### Change the random number seed and repeat the previous runs.
Change the value of ```no_experiments``` to adjust the number of times simulations will be performed. Attention! It may take a while to complete.

In [ ]:
no_experiments = 5
for _ in range(no_experiments):
    random_seed = random.uniform(1, 1000)
    print("Random Seed:", random_seed)
    simulation_1([20, 200, 1000], random_seed)
    simulation_2([15, 195, 995], random_seed)
    simulation_3([6, 186, 986], random_seed)


#### Visualization of statistics for the simulation #1.

In [ ]:
def plot(x, y):
    plt.ylim(ymax = max(y) + sum(y)/len(y), ymin = min(y) - sum(y)/len(y))
    plt.plot(x, y, 'b')
    plt.show()
    
def plot_(title, xlabel, ylabel, x, y):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plot(x, y)


You can perform the simulation #1 with any amount of jobs you want. You can modify the ```no_jobs``` to change the number of exiting jobs. Uncomment the first line if you want to see the output. Aim of this experiment to see the fluctuations in the statistical outcome by keeping the number of exiting jobs constant and running several experiments with varying random seeds.

In [ ]:
%%capture
no_jobs = 100
jobs = [no_jobs] * 10
simulation_1(jobs =jobs, random_seed = [random.uniform(1, 1000) for _ in range(len(jobs))])
no_experiment  = list(range(0, len(jobs)))

Visualize the statistical outcomes.

In [ ]:
plot_("Simulation with {} exiting jobs - Average sojourn time".format(no_jobs), "Experiment ID", "Average sojourn time", no_experiment, sojourn)
plot_("Simulation with {} exiting jobs - Average reneging rates".format(no_jobs), "Experiment ID", "Renege rate",  no_experiment, renege)
plot_("Simulation with {} exiting jobs - Proportion of time server was blocked".format(no_jobs), "Experiment ID", "Proportion of time server was blocked",no_experiment, block)
plot_("Simulation with {} exiting jobs - Average station #1 service time".format(no_jobs), "Experiment ID", "Average Service Time", no_experiment,mean_station1_service)
plot_("Simulation with {} exiting jobs - Average station #2 service time".format(no_jobs), "Experiment ID", "Average Service Time", no_experiment,mean_station2_service)
plot_("Simulation with {} exiting jobs - Average interarrival time".format(no_jobs), "Experiment ID", "Average Interarrival Time", no_experiment,mean_interarrival)
plot_("Simulation with {} exiting jobs - Average renege time".format(no_jobs), "Experiment ID", "Average renege time",  no_experiment,renege_time)
plot_("Simulation with {} exiting jobs - Average waiting time in queue".format(no_jobs), "Experiment ID", "Average waiting time",  no_experiment,mean_queue_wait)